In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
files = dbutils.fs.ls("/Volumes/workspace/default/arquivos/")
display(files)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS FORMULA1

In [0]:
database_name = "FORMULA1"

for file in dbutils.fs.ls("/Volumes/workspace/default/arquivos/"):
    path = file.path
    file_name = file.name.replace(".csv", "")
    df = spark.read.option("header", True).option("inferSchema", True).csv(path)
    df = df.withColumn("bronze_ingestion", current_timestamp())
    df.write.format("delta").saveAsTable(f"{database_name}.{file_name}")

In [0]:
%sql
SHOW TABLES IN FORMULA1

In [0]:
#exemplo com Stream table
# df_baixa_stream = (
#     spark.readStream.format("delta")
#     .table("lastros.bronze_baixa")
#     .filter(f.col("id_operacao").isin(operacoes_sem_vencimento_na_baixa))
#     .distinct()
# )
# df.write.format("delta").mode("append") \
#         .option("mergeSchema", "true") \
#         .option("optimizeWrite", "true") \
#         .option("autoCompact", "true") \
#         .saveAsTable("lastros.silver_estoque_baixa_gestora")
# df_baixa_stream.writeStream \
#     .foreachBatch(silver_estoque_baixa_gestora) \
#     .option("checkpointLocation", "/tmp/silver_estoque_baixa_gestora") \
#     .trigger(availableNow=True) \
#     .start() \
#     .awaitTermination()